In [1]:
# !git clone https://github.com/cg123/mergekit.git
# %cd mergekit
# %pip install -e .

In [3]:
import os
# os.chdir('/dtu/blackhole/02/186978/cache') # Sofie
os.chdir('/dtu/blackhole/06/187238/cache') # William
print("Current working directory:", os.getcwd())

Current working directory: /dtu/blackhole/06/187238/cache


In [4]:
OUTPUT_PATH = "hub/models--merge--MathBio--LoRA--7B"  # folder to store the result in
LORA_MERGE_CACHE = "../mergekit/merge_cache"  # change if you want to keep these for some reason
CONFIG_YML = "../mergekit/examples/math-bio-LoRA.yml"  # merge configuration file
COPY_TOKENIZER = True  # you want a tokenizer? yeah, that's what i thought
LAZY_UNPICKLE = False  # experimental low-memory model loader
LOW_CPU_MEMORY = False  # enable if you somehow have more VRAM than RAM+swap

In [3]:
# actually do merge
import torch
import yaml

from mergekit.config import MergeConfiguration
from mergekit.merge import MergeOptions, run_merge

with open(CONFIG_YML, "r", encoding="utf-8") as fp:
    merge_config = MergeConfiguration.model_validate(yaml.safe_load(fp))

run_merge(
    merge_config,
    out_path=OUTPUT_PATH,
    options=MergeOptions(
        lora_merge_cache=LORA_MERGE_CACHE,
        cuda=torch.cuda.is_available(),
        copy_tokenizer=COPY_TOKENIZER,
        lazy_unpickle=LAZY_UNPICKLE,
        low_cpu_memory=LOW_CPU_MEMORY,
    ),
)
print("Done!")

/zhome/98/5/187238/MergeKit_Env/lib64/python3.9/site-packages/networkx/utils/backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))
Warmup loader cache:   0%|                                                                                                         | 0/3 [00:00<?, ?it/s]

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

Executing graph: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 1748/1748 [12:14<00:00,  2.38it/s]


Done!


In [15]:
# Turn peft adapter model into a transformers model and save it
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

model_name = "models--MistralInstruct--MathLoRA--7B--v0.1"

# Local path, check post scriptum for explanation
model_id = f"./hub/{model_name}/checkpoint-7125"
peft_model = AutoPeftModelForCausalLM.from_pretrained(model_id)
print(type(peft_model))

merged_model = peft_model.merge_and_unload()
# The adapters are merged now and it is transformers class again
print(type(merged_model))

merged_model.save_pretrained(f'./hub/{model_name}--unloaded')

# Load and save tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)  # or use the base model directory
tokenizer.save_pretrained(f'./hub/{model_name}--unloaded')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<class 'peft.peft_model.PeftModelForCausalLM'>
<class 'transformers.models.mistral.modeling_mistral.MistralForCausalLM'>


('./hub/models--MistralInstruct--MathLoRA--7B--v0.1--unloaded/tokenizer_config.json',
 './hub/models--MistralInstruct--MathLoRA--7B--v0.1--unloaded/special_tokens_map.json',
 './hub/models--MistralInstruct--MathLoRA--7B--v0.1--unloaded/tokenizer.model',
 './hub/models--MistralInstruct--MathLoRA--7B--v0.1--unloaded/added_tokens.json',
 './hub/models--MistralInstruct--MathLoRA--7B--v0.1--unloaded/tokenizer.json')

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from datasets import Dataset, load_dataset
import json
import sys

In [9]:
model_path = "models--merge--MathBio--LoRA--7B"
model = AutoModelForCausalLM.from_pretrained(f"./hub/{model_path}")
tokenizer = AutoTokenizer.from_pretrained(f"./hub/{model_path}")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
# Define the prompt
prompt = """
Please answer the following question strictly in JSON format. Your response must contain only a JSON object 
with the keys "Explanation" and "Answer_choice". The "Answer_choice" should be one of three options: "A", "B", or "C". 
No extra text or explanation outside the JSON format is allowed.

Question: if x^2=9, then what is x?
A: 1
B: 2
C: 3
D: 5

Answer:

Output 'A', 'B', 'C', or 'D'. Full answer not needed.
"""
sent = tokenizer([prompt], return_tensors="pt")
output = model.generate(**sent, max_new_tokens=30, do_sample=True, pad_token_id=tokenizer.eos_token_id)
print(tokenizer.batch_decode(output)[0])

<s> 
Please answer the following question strictly in JSON format. Your response must contain only a JSON object 
with the keys "Explanation" and "Answer_choice". The "Answer_choice" should be one of three options: "A", "B", or "C". 
No extra text or explanation outside the JSON format is allowed.

Question: if x^2=9, then what is x?
A: 1
B: 2
C: 3
D: 5

Answer:

Output 'A', 'B', 'C', or 'D'. Full answer not needed.
```
{
  "Answer_choice": "A"
}
```</s>


In [ ]:
prompt = """
In a population of giraffes, an environmental change occurs that favors individuals that are tallest. 
As a result, more of the taller individuals are able to obtain nutrients and survive to pass along their genetic information. 
This is an example of

A. directional selection.
B. stabilizing selection.
C. sexual selection.
D. disruptive selection.

Answer:

Output 'A', 'B', 'C', or 'D'. Full answer not needed.
"""

sent = tokenizer([prompt], return_tensors="pt")
output = model.generate(**sent, max_new_tokens=30, do_sample=True, pad_token_id=tokenizer.eos_token_id)
print(tokenizer.batch_decode(output)[0])